In [ ]:
import pycaret
import missingno as msno
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pylab as plt

import os
import pandas as pd
import numpy as np
import seaborn as sns

plt.rcParams["font.family"]="Malgun Gothic"
plt.rcParams["font.size"]=12
plt.rcParams["figure.figsize"]=(14,4)
mpl.rcParams["axes.unicode_minus"]=False

import time
import statsmodels.api as sm

In [ ]:
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

In [ ]:
df=pd.read_csv('datafile.csv',encoding='ANSI')

# EDA

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=[15,7])
boxplot=df.boxplot(column=[
       'VIP', '계기비행(교육)', '계기비행(야간)', '수색(주간)', '시험비행', '주간비행',
    '계기비행(주간)', '구명벌투하', '구조(주간)', '기타', '불법조업단속', '비상절차(훈련)',
       '수색(야간)', '야간비행', '야간비행(교육)', '오염군탐색', '오염적발', '정비(행정이동)', '조명탄투하',
       '타기관지원', '합동훈련', '해상안전'],vert=0)

In [ ]:
plt.figure(figsize=[15,3])
boxplot=df.boxplot(column=['계획', '긴급', '비계획', '자체', '공군', '외주'],vert=0)

In [ ]:
plt.figure(figsize=[15,1])
boxplot=df.boxplot(column=['정비대금'],vert=0)

In [ ]:
plt.figure(figsize=[15,1])
boxplot=df.boxplot(column=['불가동일수'],vert=0)

In [ ]:
import matplotlib.pyplot as plt

df.hist(bins=50,figsize=(20,20))
plt.show()

- 이상치 제거 필요 : 정비대금, 불가동 일수

### 비행실적 시각화 위해 NaN으로 바꿈

In [ ]:
df = df.replace(0, np.NaN)

In [ ]:
msno.bar(df)

- 3년간 건수<br>
0 : 주간비행, 야간비행, 오염적발<br>
1 : 계기비행(야간), 계기비행(주간), 구명벌투하, 구조, 비상절차(훈련), 야간비행(교육), 오염군탐색, 타기관지원<br>
-> 분석에서 제거<br>

In [ ]:
df_1=df.drop(['주간비행', '야간비행', '오염적발','계기비행(야간)', '계기비행(주간)', 
              '구명벌투하', '구조(주간)', '비상절차(훈련)', '야간비행(교육)', '오염군탐색', '타기관지원'],axis=1)

In [ ]:
msno.bar(df_1)

In [ ]:
df_2 = df_1.replace(np.NaN,0)

In [ ]:
#다시 0으로 변환됐는지 확인
msno.bar(df_2)

In [ ]:
heatmap_data = df[['년도', '분기', '계획', '긴급', '비계획', '자체', '공군', '외주', '정비대금','불가동일수', 'VIP', '계기비행(교육)', '수색(주간)', 
 '시험비행','해상경비', '기타', '불법조업단속','수색(야간)','정비(행정이동)', '조명탄투하','합동훈련', '해상안전']]

In [ ]:
heatmap_data = df[['정비대금','비계획','외주','자체','해상안전','계획']]

In [ ]:
colormap = plt.cm.PuBu 
plt.figure(figsize=(10,10)) 
plt.title("Person Correlation of Features", y = 1.05, size = 15) 
sns.heatmap(heatmap_data.astype(float).corr(), linewidths = 0.1, vmax = 1.0, square = True, cmap = colormap, linecolor = "white", annot = True, annot_kws = {"size" : 16})

In [ ]:
corr_matrix=df_2.corr()
corr_matrix['정비대금'].sort_values(ascending=False)

In [ ]:
#corr>0.2 변수들로 scatter matrix

from pandas.plotting import scatter_matrix

attributes=['정비대금','비계획','외주','자체','해상안전','계획']
scatter_matrix(df_2[attributes],figsize=(12,8))

In [ ]:
sns.distplot(df['계획'],label='계획')
sns.distplot(df['긴급'],label='긴급')
sns.distplot(df['비계획'],label='비계획')
plt.xlabel('분기별 건수')
plt.ylabel('발생 빈도')
plt.legend()

In [ ]:
sns.distplot(df['자체'],label='자체')
sns.distplot(df['공군'],label='공군')
sns.distplot(df['외주'],label='외주')
plt.xlabel('분기별 건수')
plt.ylabel('발생 빈도')
plt.legend()

In [ ]:
sns.distplot(df['불가동일수'])

In [ ]:
ratio = [106,10,59]
labels = ['계획', '긴급', '비계획']
explode = [0.1,0.1,0.1]
colors = ['#ff9999', '#ffc000', '#8fd9b6']

plt.pie(ratio, labels=labels, autopct='%.1f%%', startangle=260, counterclock=False, 
        explode=explode, shadow=True,colors=colors)
plt.show()

In [ ]:
ratio = [101,19,53]
labels = ['자체', '공군', '외주']
explode = [0.1,0.1,0.1]
colors = ['#ffc000', '#8fd9b6', '#d395d0']

plt.pie(ratio, labels=labels, autopct='%.1f%%', startangle=260, counterclock=False, 
        explode=explode, shadow=True,colors=colors)
plt.show()

In [ ]:
sns.distplot(df['정비대금'],bins=60)

- EDA 끝

# 더미변수화

In [ ]:
df_2.info()

In [ ]:
df_dummy=pd.get_dummies(data=df_2,columns=['기종','호기','년도','분기'])

In [ ]:
df_dummy.head()

In [ ]:
df_dummy.info()

In [ ]:
df=df_dummy

# 변수 스케일링

In [ ]:
colnames=['정비대금',
       '불가동일수', 'VIP', '계기비행(교육)', '계기비행(야간)', '수색(주간)', '시험비행', '주간비행',
       '해상경비', '계기비행(주간)', '구명벌투하', '구조(주간)', '기타', '불법조업단속', '비상절차(훈련)',
       '수색(야간)', '야간비행', '야간비행(교육)', '오염군탐색', '오염적발', '정비(행정이동)', '조명탄투하',
       '타기관지원', '합동훈련', '해상안전']

In [ ]:
df_cat=df.iloc[:,:10]
df_fl=df.iloc[:,10:]

In [ ]:
from sklearn.model_selection import train_test_split
from pycaret.regression import *

# MinMax Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

new_colnames=[i+'_mms' for i in colnames]
df_mms=pd.concat([df_cat,pd.DataFrame(scaler.fit_transform(df_fl[colnames]),columns=new_colnames)],axis=1)

In [ ]:
train,test=train_test_split(df_mms,test_size=0.2,random_state=42)
exp1=setup(train,target='정비대금_mms',train_size=0.8)

In [ ]:
best_5=compare_models(sort='RMSE',n_select=1)

# Robust Scaler

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler=RobustScaler()

new_colnames=[i+'_rs' for i in colnames]
df_rs=pd.concat([df_cat,pd.DataFrame(scaler.fit_transform(df_fl[colnames]),columns=new_colnames)],axis=1)

In [ ]:
train,test=train_test_split(df_rs,test_size=0.2,random_state=42)
exp1=setup(train,target='정비대금_rs',train_size=0.8)

In [ ]:
best_5=compare_models(sort='RMSE',n_select=1)

# Standard Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

new_colnames=[i+'_ss' for i in colnames]
df_ss=pd.concat([df_cat,pd.DataFrame(scaler.fit_transform(df_fl[colnames]),columns=new_colnames)],axis=1)

In [ ]:
train,test=train_test_split(df_ss,test_size=0.2,random_state=42)
exp1=setup(train,target='정비대금_ss',train_size=0.8)

In [ ]:
best_5=compare_models(sort='R2',n_select=1)

In [ ]:
knn=create_model('knn',cross_validation=False) #KNN

In [ ]:
rf=create_model('rf',cross_validation=False) #RF

In [ ]:
blender_specific = blend_models(estimator_list = [rf,knn], optimize = 'R2')

# plot_model

In [ ]:
plot_model(blender_specific)

In [ ]:
plot_model(blender_specific, plot='error')

In [ ]:
plot_model(rf)

In [ ]:
plot_model(rf, plot='error')

In [ ]:
plot_model(rf, plot='feature')

In [ ]:
plot_model(knn)

In [ ]:
plot_model(knn, plot='error')

# Text Mining

In [ ]:
textfile=pd.read_csv('자체.csv',encoding='ANSI')

In [ ]:
df=textfile['정비명']

In [ ]:
df.to_csv('자체정비.txt',index=False)

In [ ]:
file=open('자체정비.txt', 'rt', encoding='UTF8')
 
text = file.read() 
 
wordList = text.split()
 
wordCount = {} 
 
for word in wordList:
 
    # Get 명령어를 통해, Dictionary에 Key가 없으면 0리턴
 
    wordCount[word] = wordCount.get(word, 0) + 1 
    
    keys = sorted(wordCount.keys())
 
for word in keys:
 
    print(word + ':' + str(wordCount[word])) 